In [1]:
%%time
import sys
sys.path.insert(0, '/Users/shuyafeng/PycharmProjects/xiaozaizi/codes/')

import os
import json
import warnings
import pprint


%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

CPU times: user 15.7 ms, sys: 13.6 ms, total: 29.3 ms
Wall time: 28.7 ms


In [4]:
import numpy as np
import scipy.io
from numpy.random import choice, laplace
from scipy.special import lambertw
from scipy.stats import laplace
from codes.kse_test import kse_test


In [52]:
#dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/server1.mat')['dataset4']
dataset4 = np.array(pd.read_csv('cv_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = np.linspace(0.1, 10, 4)
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = kse_test(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [53]:
Output

array([0.93684211, 0.21883657, 0.22770083, 0.21052632, 0.2100831 ,
       0.23789474, 0.23091413, 0.2233795 , 0.22027701, 0.21972299,
       0.18703601, 0.23800554, 0.24110803, 0.23135734, 0.21307479,
       0.23157895, 0.20232687, 0.23024931, 0.24166205, 0.21052632,
       0.22825485, 0.19390582, 0.23534626, 0.21706371, 0.22770083,
       0.22692521, 0.21307479, 0.23047091, 0.21252078, 0.96753463,
       0.21872576, 0.17772853, 0.23146814, 0.22880886, 0.23612188,
       0.24210526, 0.19523546, 0.23091413, 0.22814404, 0.18648199,
       0.96188366, 0.95722992, 0.97894737, 0.97806094, 1.        ,
       0.9731856 , 0.95213296, 0.98814404, 0.92698061, 0.94770083,
       0.98947368, 0.93927978, 0.93096953, 0.95124654, 0.99933518,
       0.92022161, 0.97861496, 0.23091413, 0.23944598, 0.94027701,
       0.24210526, 0.2265928 , 0.18127424, 0.23102493, 0.21296399,
       0.21972299, 0.53207756, 0.91878116, 0.2398892 , 0.21052632,
       0.22072022, 0.20764543, 0.2134072 , 0.21806094, 0.22105

In [57]:
EVAL = np.zeros((4, 7))
EVAL1 = np.zeros((4, 7))

for rr in range(4):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN

        noiseunif = kse_test(unif)
        noiseanom = kse_test(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.85:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.85:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.85:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL1[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [58]:
import pandas as pd

In [59]:
pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.76,9.0,0.858974,0.450000,0.409091,0.428571,5.235369
1,0.74,22.0,0.666667,0.458333,1.000000,0.628571,7.211103
2,0.84,21.0,0.807692,0.583333,0.954545,0.724138,7.754764
3,0.81,17.0,0.820513,0.548387,0.772727,0.641509,7.032393
